In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import csv
import dateutil
%pylab inline

Populating the interactive namespace from numpy and matplotlib


###Complaints Data (Manhattan, 2015)

In [2]:
complaints = pd.read_csv('Housing_Maintenance_Code_Complaints.csv', parse_dates=['ReceivedDate', 'StatusDate'])

In [3]:
complaints = complaints[complaints['BoroughID']==1]
complaints['ReceivedYR'] = complaints['ReceivedDate'].map(lambda x: x.year)
complaints_2015 =  complaints[complaints['ReceivedYR']==2015]

###The complaints data of Manhattan, 2015 are _"88,118"_

In [4]:
print len(complaints_2015)
print complaints_2015.dtypes

88118
ComplaintID                int64
BuildingID                 int64
BoroughID                  int64
Borough                   object
HouseNumber               object
StreetName                object
Zip                      float64
Block                      int64
Lot                        int64
Apartment                 object
CommunityBoard             int64
ReceivedDate      datetime64[ns]
StatusID                   int64
Status                    object
StatusDate        datetime64[ns]
ReceivedYR                 int64
dtype: object


In [5]:
#complaints_2015.sort(['ReceivedDate'])[:3]

### New columns ['BBL', 'Full_Address'] to match data in building level or BBL level
__LOGIC__
* BBL = BoroughID + Block + Lot
* Full_Address = HouseNumber + StreeName + 'New York, NY' + Zip

#Don't worry about the python warning (red box) which is just copy and paste columns issue. 

In [6]:
#Covert dtype int to str
complaints_2015['BoroughID'] = complaints_2015['BoroughID'].astype(str)
complaints_2015['Block'] = complaints_2015['Block'].astype(str)
complaints_2015['Lot'] = complaints_2015['Lot'].astype(str)
complaints_2015['Zip'] = complaints_2015['Zip'].astype(str)
complaints_2015['Zip'] = complaints_2015['Zip'].map(lambda x: str(x)[0:5])

#Add leading zeros to the Block and Lot (to make 10 digits BBL)
complaints_2015['Block'] = complaints_2015['Block'].apply(lambda x: x.zfill(5))
complaints_2015['Lot'] = complaints_2015['Lot'].apply(lambda x: x.zfill(4))

#Combime columns 
complaints_2015['BBL'] = complaints_2015['BoroughID'] + complaints_2015['Block'] + complaints_2015['Lot']
complaints_2015['Full_Address'] = complaints_2015['HouseNumber'] + ' ' + complaints_2015['StreetName'] + ' New York, NY ' + complaints_2015['Zip']

C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [7]:
print complaints_2015.Full_Address[:3]

174927                    442 2 AVENUE New York, NY 10010
174928    2155 ADAM C POWELL BOULEVARD New York, NY 10027
174929           1528 AMSTERDAM AVENUE New York, NY 10031
Name: Full_Address, dtype: object


###Geocoding for address matching in Bldg level (daily limitation --> will figure out)

In [8]:
#import geocoder

# Function to send address to geo-encode from Google API
#def geoencode(address):
    #g = geocoder.google(address) # ex. 30-27 Greenpoint Avenue LIC, NY
    #loc = [g.lat, g.lng, g.geometry] # create a list
    #return loc

#address = []
#lat = []
#lng = []
#geometry = []

#for i in complaints_2015.Full_Address:
    #g = geoencode(i) # call geoencode function
    #address.append(i)
    #lat.append(g[:1])
    #lng.append(g[1:2])
    #geometry.append(g[2:3])

#print "Finished Geocoding"

#Lat = pd.DataFrame(lat)
#Lat.columns=['Lat']
#Lon = pd.DataFrame(lng)
#Lon.columns=['Lon']
#Geometry = pd.DataFrame(geometry)
#Geometry.columns=['Geometry']

#complaints_2015 = pd.concat([complaints_2015, Lat, Lon, Geometry], axis=1)
#result.to_csv('e_waste_dataset_recyclingfacilities_geocoding_nyc.csv')

###Violations Data (Manhattan, 2015)

In [9]:
violations = pd.read_csv('Housing_Maintenance_Code_Violations.csv', parse_dates=['InspectionDate', 'ApprovedDate']) # violations

In [10]:
violations = violations[violations['BoroID']==1]
violations['InspectionYR'] = violations['InspectionDate'].map(lambda x: x.year)
violations_2015 =  violations[violations['InspectionYR']==2015]

###The violations data of Manhattan, 2015 are _"112,574"_
### !!! violations - complaints = 112,574 - 88,118 = 24,456

In [11]:
print len(violations_2015)
print violations_2015.dtypes

112574
ViolationID                       int64
BuildingID                        int64
RegistrationID                    int64
BoroID                            int64
Boro                             object
HouseNumber                      object
LowHouseNumber                   object
HighHouseNumber                  object
StreetName                       object
StreetCode                        int64
Zip                             float64
Apartment                        object
Story                            object
Block                             int64
Lot                               int64
Class                            object
InspectionDate           datetime64[ns]
ApprovedDate             datetime64[ns]
OriginalCertifyByDate            object
OriginalCorrectByDate            object
NewCertifyByDate                 object
NewCorrectByDate                 object
CertifiedDate                    object
OrderNumber                       int64
NOVID                           f

### New columns ['BBL', 'Full_Address'] to match data in building level or BBL level
__LOGIC__
* BBL = BoroID + Block + Lot
* Full_Address = HouseNumber + StreeName + 'New York, NY' + Zip

#Don't worry about the python warning (red box) which is just copy and paste columns issue. 

In [12]:
#Covert dtype int to str
violations_2015['BoroID'] = violations_2015['BoroID'].astype(str)
violations_2015['Block'] = violations_2015['Block'].astype(str)
violations_2015['Lot'] = violations_2015['Lot'].astype(str)
violations_2015['Zip'] = violations_2015['Zip'].astype(str)
violations_2015['Zip'] = violations_2015['Zip'].map(lambda x: str(x)[0:5])

#Add leading zeros to the Block and Lot (to make 10 digits BBL)
violations_2015['Block'] = violations_2015['Block'].apply(lambda x: x.zfill(5))
violations_2015['Lot'] = violations_2015['Lot'].apply(lambda x: x.zfill(4))

#Combime columns 
violations_2015['BBL'] = violations_2015['BoroID'] + violations_2015['Block'] + violations_2015['Lot']
violations_2015['Full_Address'] = violations_2015['HouseNumber'] + ' ' + violations_2015['StreetName'] + ' New York, NY ' + violations_2015['Zip']

C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Boyeong\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [13]:
print violations_2015[:3]

        ViolationID  BuildingID  RegistrationID BoroID       Boro HouseNumber  \
393664     10506902          14          115401      1  MANHATTAN        1058   
393665     10506854          14          115401      1  MANHATTAN        1058   
393666     10506855          14          115401      1  MANHATTAN        1058   

       LowHouseNumber HighHouseNumber StreetName  StreetCode  \
393664           1058            1064   1 AVENUE       10010   
393665           1058            1064   1 AVENUE       10010   
393666           1058            1064   1 AVENUE       10010   

                      ...                OrderNumber    NOVID  \
393664                ...                        617  4983871   
393665                ...                        702  4983870   
393666                ...                       1503  4983870   

                                           NOVDescription NOVIssuedDate  \
393664  Â§ 27-2056.6 ADM CODE - CORRECT THE LEAD-BASED...    01/15/2015   
393665 

In [14]:
#save as csv (complaints)
#complaints_2015.to_csv('complaints_2015.csv')

In [15]:
#save as csv (violations)
#violations_2015.to_csv('violations_2015.csv')

In [16]:
print complaints_2015.sort(['ReceivedDate'])[:10]

        ComplaintID  BuildingID BoroughID    Borough HouseNumber  \
222949      7185130       25378         1  MANHATTAN          58   
206477      7184758       36448         1  MANHATTAN         166   
212390      7184558       41291         1  MANHATTAN         142   
209506      7184743       25378         1  MANHATTAN          58   
182764      7185058        8077         1  MANHATTAN        3580   
202040      7184525        5036         1  MANHATTAN        3854   
182590      7184738       42077         1  MANHATTAN         543   
176117      7185364       43644         1  MANHATTAN         558   
190972      7185277       25742         1  MANHATTAN          70   
217899      7184889        9749         1  MANHATTAN         380   

                StreetName    Zip  Block   Lot Apartment  CommunityBoard  \
222949  MARBLE HILL AVENUE  10463  02215  0555      BLDG               8   
206477      WEST 87 STREET  10024  01217  0057      BLDG               7   
212390     WEST 139 STR

### Just take an example '166 WEST 87 STREET New York, NY 10024'

In [17]:
print complaints_2015.ReceivedDate[complaints_2015['Full_Address']=='166 WEST 87 STREET New York, NY 10024']
print '\n'
print violations_2015.InspectionDate[violations_2015['Full_Address']=='166 WEST 87 STREET New York, NY 10024']

177538   2015-01-02
180760   2015-01-02
182051   2015-01-02
184090   2015-01-01
190575   2015-01-07
194045   2015-01-01
194994   2015-01-02
197891   2015-01-02
202922   2015-01-17
206477   2015-01-01
207383   2015-01-01
212182   2015-01-01
212905   2015-01-01
213611   2015-01-06
222076   2015-01-02
223331   2015-01-20
230521   2015-01-01
235693   2015-02-13
238425   2015-02-13
246935   2015-02-14
251911   2015-02-14
253054   2015-02-28
261458   2015-02-14
265719   2015-02-01
268025   2015-02-01
287692   2015-02-28
294979   2015-02-12
302372   2015-03-13
306320   2015-03-09
307602   2015-03-09
            ...    
362671   2015-05-03
375160   2015-06-17
376460   2015-06-16
378155   2015-06-27
380253   2015-06-27
381184   2015-06-01
387450   2015-06-26
388277   2015-07-14
390257   2015-07-09
403787   2015-07-10
433094   2015-09-08
435908   2015-09-30
436614   2015-10-17
438867   2015-10-17
439289   2015-10-17
442064   2015-10-19
444469   2015-10-17
455186   2015-10-17
455880   2015-10-17


###Data group by same address to compare C vs. V (simle numeric gap)

In [18]:
#count of complaints and violations on same address (To compare the number of gap)
c_a = complaints_2015['Full_Address'].value_counts().to_frame()
v_a = violations_2015['Full_Address'].value_counts().to_frame()

In [19]:
c_a.columns =['Complaints']
v_a.columns = ['Violations']
print c_a.head()
print v_a.head()
print '\n'
print len(c_a)
print len(v_a)

                                            Complaints
34 ARDEN STREET New York, NY 10040                2167
859 9 AVENUE New York, NY 10019                    453
344 WEST 72 STREET New York, NY 10023              383
101 POST AVENUE New York, NY 10034                 343
80 FT WASHINGTON AVENUE New York, NY 10032         338
                                          Violations
560 AUDUBON AVENUE New York, NY 10040            498
3851 BROADWAY New York, NY 10032                 336
344 WEST 72 STREET New York, NY 10023            321
1792 AMSTERDAM AVENUE New York, NY 10031         287
203 WEST 145 STREET New York, NY 10039           273


10960
16206


###The number of buildings occuring complaints are "10,960"
###The number of buildings occuring violations are "16,206"
#Now, join the data set to compare COMPLAINTS vs. VIOLATIONS

In [20]:
#Join data C vs. V
c_v_a = pd.concat([c_a, v_a], axis=1)
len(c_v_a)

19439

### --> 19,439 buildings have either complaints or violations
# GAP calculation

In [26]:
#Calculate gap between complaints and violations 
#Positive no. means higher complaints but less violateions (may be good reporting?)
#Negative no. means less complaints but higher violations 
c_v_a = c_v_a.fillna(0)
c_v_a['Gap']=c_v_a['Complaints'] - c_v_a['Violations']

In [41]:
len(c_v_a[(c_v_a.Complaints>0) & (c_v_a.Violations>0)])

7727

### --> 7,727 buildings have both complaints and violations (at least one reporting or inspection)

In [39]:
c_v_a.head()

,Complaints,Violations,Gap
"1 5 AVENUE New York, NY 10003",1,3,-2
"1 ADRIAN AVENUE New York, NY 10463",54,52,2
"1 AUDUBON AVENUE New York, NY 10032",2,0,2
"1 BENNETT AVENUE New York, NY 10033",41,12,29
"1 BOGARDUS PLACE New York, NY 10040",47,23,24


#Top buildings with huge gap

In [29]:
c_v_a.sort(['Gap'])[:30]

,Complaints,Violations,Gap
"560 AUDUBON AVENUE New York, NY 10040",93,498,-405
"203 WEST 145 STREET New York, NY 10039",26,273,-247
"1792 AMSTERDAM AVENUE New York, NY 10031",57,287,-230
"772 ST NICHOLAS AVENUE New York, NY 10031",64,269,-205
"620 WEST 141 STREET New York, NY 10031",48,251,-203
"85 BOWERY New York, NY 10002",21,216,-195
"518 WEST 139 STREET New York, NY 10031",56,241,-185
"640 RIVERSIDE DRIVE New York, NY 10031",30,190,-160
"1619 AMSTERDAM AVENUE New York, NY 10031",51,211,-160
"135 WEST 145 STREET New York, NY 10039",49,204,-155


In [42]:
#save as csv (gaps)
#c_v_a.to_csv('c_v_a_gap_2015.csv')

#What I (we) want to do next...
* Plot timeseries
* Geocoding (point mapping + kernel mapping)
*